In [ ]:
# Install any required packages (Colab only)
#!pip install geohash2 geopandas datascience shapely pandas matplotlib seaborn scikit-learn


Defaulting to user installation because normal site-packages is not writeable
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
     --------------------------------------- 11.1/11.1 MB 19.9 MB/s eta 0:00:00
     -------------------------------------- 301.8/301.8 kB 4.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Traffic & Air Quality Analysis Using datascience + GeoPandas + DBSCAN + Geohashing

# STEP 1: Import libraries
# ----------------------------------------
# Import all necessary libraries for data handling, spatial analysis, plotting, and clustering
from datascience import Table
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import geohash2
import glob

In [3]:
# STEP 2: Load and combine AIR QUALITY data from multiple CSVs
# ----------------------------------------
# Combine all air quality data parts into one DataFrame
files = sorted(glob.glob("chicago_eclipse_data_part_*.csv"))
aq_df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
aq_df = aq_df.drop(columns=[
    "City",
    "DeviceId",
    "LocationName",
    "Temperature",
    "Humidity",
    "BatteryLevel",
    "PercentBattery",
    "CellSignal"
])
print(aq_df.columns)

Index(['Latitude', 'Longitude', 'ReadingDateTimeUTC', 'PM25', 'CalibratedPM25',
       'CalibratedO3', 'CalibratedNO2', 'CO'],
      dtype='object')


In [9]:
aq_df

,Latitude,Longitude,ReadingDateTimeUTC,PM25,CalibratedPM25,CalibratedO3,CalibratedNO2,CO
0,41.794921,-87.625857,2021-06-20 00:03:00+00:00,5.561094,NaN,NaN,NaN,0.123580
1,41.794921,-87.625857,2021-06-20 00:08:10+00:00,6.633914,NaN,NaN,NaN,0.132103
2,41.794921,-87.625857,2021-06-20 00:13:20+00:00,4.068707,NaN,NaN,NaN,0.131126
3,41.794921,-87.625857,2021-06-20 00:18:30+00:00,6.351702,NaN,NaN,NaN,0.138784
4,41.794921,-87.625857,2021-06-20 00:23:40+00:00,9.574065,NaN,NaN,NaN,0.413070
...,...,...,...,...,...,...,...,...
2461084,41.903627,-87.643443,2021-08-20 11:55:10+00:00,17.235613,15.85,22.52,13.78,0.407807
2461085,41.903627,-87.643443,2021-08-20 12:00:20+00:00,11.742577,14.20,20.00,12.48,0.382384
2461086,41.903627,-87.643443,2021-08-20 12:05:30+00:00,13.482563,14.78,21.90,13.01,0.385963
2461087,41.903627,-87.643443,2021-08-20 12:10:39+00:00,10.450826,15.47,19.72,13.55,0.438555


In [8]:
# Convert timestamp to datetime format
aq_df["ReadingDateTimeUTC"] = pd.to_datetime(aq_df["ReadingDateTimeUTC"], utc=True)

In [10]:
# Create geometry column for air quality sensor locations
aq_df["geometry"] = aq_df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
gdf_aq = gpd.GeoDataFrame(aq_df, geometry="geometry", crs="EPSG:4326")

In [6]:
# STEP 3: Load TAXI data
# ----------------------------------------
# Load taxi trip data from a single file
taxi_df = pd.read_csv("taxi_data.csv")

# Keep only relevant columns
columns_to_keep = [
    "Trip Start Timestamp",
    "Trip End Timestamp",
    "Trip Seconds",
    "Trip Miles",
    "Pickup Centroid Latitude",
    "Pickup Centroid Longitude"
]

taxi_df = taxi_df[columns_to_keep]

print(taxi_df.columns)

Index(['Trip Start Timestamp', 'Trip End Timestamp', 'Trip Seconds',
       'Trip Miles', 'Pickup Centroid Latitude', 'Pickup Centroid Longitude'],
      dtype='object')


In [7]:
taxi_df

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Centroid Latitude,Pickup Centroid Longitude
0,10/03/2021 08:30:00 PM,10/03/2021 09:00:00 PM,1584.0,18.07,41.877406,-87.621972
1,10/03/2021 08:30:00 PM,10/03/2021 09:00:00 PM,1606.0,16.60,41.979071,-87.903040
2,10/03/2021 08:30:00 PM,10/03/2021 08:45:00 PM,660.0,1.40,41.899602,-87.633308
3,10/03/2021 08:30:00 PM,10/03/2021 08:30:00 PM,498.0,3.45,41.980264,-87.913625
4,10/03/2021 08:30:00 PM,10/03/2021 08:45:00 PM,618.0,0.00,41.878866,-87.625192
...,...,...,...,...,...,...
2618446,01/01/2021 08:45:00 PM,01/01/2021 09:15:00 PM,1500.0,11.90,41.707311,-87.534903
2618447,01/01/2021 08:45:00 PM,01/01/2021 09:00:00 PM,807.0,4.21,41.953582,-87.723452
2618448,01/01/2021 08:45:00 PM,01/01/2021 09:00:00 PM,1260.0,0.00,41.792592,-87.769615
2618449,01/01/2021 08:45:00 PM,01/01/2021 09:15:00 PM,1320.0,8.90,41.835118,-87.618678


In [13]:
print("🔍 Null values in Air Quality Data:")
print(aq_df.isnull().sum())

print("🔍 Null values in Taxi Data:")
print(taxi_df.isnull().sum())

def null_report(df, name):
    print(f"🧾 Missing values in {name}:")
    nulls = df.isnull().sum()
    percent = (nulls / len(df)) * 100
    report = pd.DataFrame({"Null Count": nulls, "Percent": percent.round(2)})
    print(report[report["Null Count"] > 0])  # Show only columns with nulls
    print()

null_report(aq_df, "Air Quality Data")
null_report(taxi_df, "Taxi Data")


🔍 Null values in Air Quality Data:
Latitude              0
Longitude             0
ReadingDateTimeUTC    0
PM25                  0
CalibratedPM25        0
CalibratedO3          0
CalibratedNO2         0
CO                    0
geometry              0
dtype: int64
🔍 Null values in Taxi Data:
Trip Start Timestamp         0
Trip End Timestamp           0
Trip Seconds                 0
Trip Miles                   0
Pickup Centroid Latitude     0
Pickup Centroid Longitude    0
dtype: int64
🧾 Missing values in Air Quality Data:
Empty DataFrame
Columns: [Null Count, Percent]
Index: []

🧾 Missing values in Taxi Data:
Empty DataFrame
Columns: [Null Count, Percent]
Index: []



In [12]:
# Drop all rows with any NaN values in air quality data
aq_df = aq_df.dropna()
# Drop all rows with any NaN values in taxi data
taxi_df = taxi_df.dropna()


In [15]:
taxi_df = taxi_df.copy()
# Convert pickup and dropoff timestamps to datetime format
taxi_df["Trip Start Timestamp"] = pd.to_datetime(
    taxi_df["Trip Start Timestamp"],
    format="%m/%d/%Y  %I:%M:%S %p",
    utc=True
)
taxi_df["Trip End Timestamp"] = pd.to_datetime(
    taxi_df["Trip End Timestamp"],
    format="%m/%d/%Y  %I:%M:%S %p",
    utc=True
)

In [16]:
taxi_df

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Centroid Latitude,Pickup Centroid Longitude
0,2021-10-03 20:30:00+00:00,2021-10-03 21:00:00+00:00,1584.0,18.07,41.877406,-87.621972
1,2021-10-03 20:30:00+00:00,2021-10-03 21:00:00+00:00,1606.0,16.60,41.979071,-87.903040
2,2021-10-03 20:30:00+00:00,2021-10-03 20:45:00+00:00,660.0,1.40,41.899602,-87.633308
3,2021-10-03 20:30:00+00:00,2021-10-03 20:30:00+00:00,498.0,3.45,41.980264,-87.913625
4,2021-10-03 20:30:00+00:00,2021-10-03 20:45:00+00:00,618.0,0.00,41.878866,-87.625192
...,...,...,...,...,...,...
2618446,2021-01-01 20:45:00+00:00,2021-01-01 21:15:00+00:00,1500.0,11.90,41.707311,-87.534903
2618447,2021-01-01 20:45:00+00:00,2021-01-01 21:00:00+00:00,807.0,4.21,41.953582,-87.723452
2618448,2021-01-01 20:45:00+00:00,2021-01-01 21:00:00+00:00,1260.0,0.00,41.792592,-87.769615
2618449,2021-01-01 20:45:00+00:00,2021-01-01 21:15:00+00:00,1320.0,8.90,41.835118,-87.618678


In [17]:
# STEP 4: Clean and engineer features
# ----------------------------------------
# Filter out rows without spatial pickup info and compute trip metrics
taxi_df = taxi_df.dropna(subset=["Pickup Centroid Latitude", "Pickup Centroid Longitude"])
taxi_df["trip_minutes"] = taxi_df["Trip Seconds"] / 60

taxi_df["avg_speed_mph"] = taxi_df["Trip Miles"] / (taxi_df["trip_minutes"] / 60)

In [31]:
# Create geometry column for pickup locations
taxi_df["geometry"] = taxi_df.apply(lambda row: Point(row["Pickup Centroid Longitude"], row["Pickup Centroid Latitude"]), axis=1)
gdf_taxi = gpd.GeoDataFrame(taxi_df, geometry="geometry", crs="EPSG:4326")

In [32]:
taxi_df.to_csv("cleaned_taxi_data.csv", index=False)
gdf_aq.to_csv("cleaned_air_quality_data.csv", index=False)

In [33]:
# METHOD A: SPATIAL JOIN USING BUFFER (Precise Proximity)
# ----------------------------------------
gdf_taxi_buffer = gdf_taxi.to_crs(epsg=3857)
gdf_aq_buffer = gdf_aq.to_crs(epsg=3857)
gdf_aq_buffer["geometry"] = gdf_aq_buffer.buffer(200)  # 200m buffer around each sensor

In [34]:
# Find taxi pickups within buffer zone of air sensors
joined_buffer = gpd.sjoin(gdf_taxi_buffer, gdf_aq_buffer, how="inner", predicate="within")

In [37]:
joined_buffer.to_csv("spatial_join_data.csv", index=False)

In [42]:
def safe_merge_asof(left_df, right_df, on_left="Trip Start Timestamp", on_right="ReadingDateTimeUTC", batch_size=10000):
    """
    Performs merge_asof in batches to avoid MemoryError.
    """
    result_batches = []
    right_sorted = right_df.sort_values(on_right)  # Sort right side once

    for start in range(0, len(left_df), batch_size):
        end = start + batch_size
        batch = left_df.iloc[start:end].copy()

        batch_sorted = batch.sort_values(on_left)

        merged = pd.merge_asof(
            batch_sorted,
            right_sorted,
            left_on=on_left,
            right_on=on_right,
            direction="nearest",
            tolerance=pd.Timedelta("15min")
        )
        result_batches.append(merged)

    print("✅ Batching merge completed.")
    return pd.concat(result_batches, ignore_index=True)


ERROR! Session/line number was not unique in database. History logging moved to new session 23


In [43]:
gdf_aq_buffer = gdf_aq_buffer.sort_values("ReadingDateTimeUTC")

joined_buffer = safe_merge_asof(joined_buffer, gdf_aq_buffer)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Abdullah\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3667, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Abdullah\AppData\Local\Temp\ipykernel_21040\2587913881.py", line 3, in <module>
    joined_buffer = safe_merge_asof(joined_buffer, gdf_aq_buffer)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Abdullah\AppData\Local\Temp\ipykernel_21040\3966413573.py", line 10, in safe_merge_asof
    batch = left_df.iloc[start:end].copy()
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Abdullah\AppData\Roaming\Python\Python311\site-packages\geopandas\geodataframe.py", line 1826, in copy
    copied = super().copy(deep=deep)
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Abdullah\AppData\Roaming\Python\Python311\site-packages\pandas\core\generic.py", line 6808, in copy
    data = self._mgr.copy(deep=deep)
           ^

In [38]:
# Match each taxi pickup to closest-in-time sensor reading within 15 mins
joined_buffer = joined_buffer.sort_values("ReadingDateTimeUTC")
joined_buffer = pd.merge_asof(
    joined_buffer.sort_values("Trip Start Timestamp"),
    gdf_aq_buffer.sort_values("ReadingDateTimeUTC"),
    left_on="Trip Start Timestamp",
    right_on="ReadingDateTimeUTC",
    direction="nearest",
    tolerance=pd.Timedelta("15min")
)

MemoryError: Unable to allocate 9.71 GiB for an array with shape (6, 217297289) and data type float64

In [ ]:
# Aggregate by hour
joined_buffer["hour"] = joined_buffer["ReadingDateTimeUTC"].dt.floor("H")
agg_buffer = joined_buffer.groupby("hour").agg({
    "avg_speed_mph": "mean",
    "trip_miles": "sum",
    "trip_seconds": "count",
    "CalibratedPM25": "mean"
}).reset_index()

In [18]:
# METHOD B: GEOHASH JOIN (Grid-based Fast Approximation)
# ----------------------------------------
# Convert lat/lon to geohash codes
gdf_aq["geohash"] = gdf_aq.geometry.apply(lambda p: geohash2.encode(p.y, p.x, precision=6))
gdf_taxi["geohash"] = gdf_taxi.geometry.apply(lambda p: geohash2.encode(p.y, p.x, precision=6))

NameError: name 'gdf_taxi' is not defined

In [ ]:
# Join on geohash codes
joined_hash = pd.merge(gdf_taxi, gdf_aq, on="geohash", how="inner")

In [ ]:
# Match nearest-in-time sensor reading
joined_hash = joined_hash.sort_values("ReadingDateTimeUTC")
joined_hash = pd.merge_asof(
    joined_hash.sort_values("trip_start_timestamp"),
    gdf_aq.sort_values("ReadingDateTimeUTC"),
    left_on="trip_start_timestamp",
    right_on="ReadingDateTimeUTC",
    direction="nearest",
    tolerance=pd.Timedelta("15min")
)

In [ ]:
# Aggregate by hour
joined_hash["hour"] = joined_hash["ReadingDateTimeUTC"].dt.floor("H")
agg_hash = joined_hash.groupby("hour").agg({
    "avg_speed_mph": "mean",
    "trip_miles": "sum",
    "trip_seconds": "count",
    "CalibratedPM25": "mean"
}).reset_index()

In [ ]:
# COMPARISON VISUALIZATION
# ----------------------------------------
plt.figure(figsize=(12, 6))
plt.plot(agg_buffer["hour"], agg_buffer["CalibratedPM25"], label="Buffer Method")
plt.plot(agg_hash["hour"], agg_hash["CalibratedPM25"], label="Geohash Method")
plt.title("PM2.5 Over Time: Buffer vs Geohash Join")
plt.xlabel("Hour")
plt.ylabel("PM2.5")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# SELECT AGGREGATION METHOD FOR DEEPER ANALYSIS
# ----------------------------------------
agg = agg_buffer.copy()

In [ ]:
# HISTOGRAMS
# ----------------------------------------
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.hist(agg["CalibratedPM25"].dropna(), bins=20, edgecolor='black')
plt.title("Histogram of Calibrated PM2.5")
plt.xlabel("PM2.5")
plt.ylabel("Frequency")

plt.subplot(1, 2, 2)
plt.hist(agg["avg_speed_mph"].dropna(), bins=20, edgecolor='black')
plt.title("Histogram of Average Speed")
plt.xlabel("Speed (mph)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

In [ ]:
# SCATTERPLOTS WITH REGRESSION LINES
# ----------------------------------------
sns.lmplot(data=agg, x="avg_speed_mph", y="CalibratedPM25")
plt.title("Speed vs PM2.5\nFaster traffic is often associated with lower PM2.5 levels. Slower speeds may indicate congestion and poor air quality.")
plt.xlabel("Average Speed (mph)")
plt.ylabel("Calibrated PM2.5")
plt.show()

sns.lmplot(data=agg, x="trip_seconds", y="CalibratedPM25")
plt.title("Trip Count vs PM2.5\nHigher trip counts can indicate increased congestion, possibly elevating air pollution levels.")
plt.xlabel("Trip Count")
plt.ylabel("Calibrated PM2.5")
plt.show()

In [ ]:
# DBSCAN CLUSTERING
# ----------------------------------------
features = agg[["avg_speed_mph", "CalibratedPM25"]].dropna()
scaler = StandardScaler()
scaled = scaler.fit_transform(features)
db = DBSCAN(eps=0.5, min_samples=5).fit(scaled)
agg["cluster"] = -1
agg.loc[features.index, "cluster"] = db.labels_

In [ ]:
# CLUSTER VISUALIZATION
# ----------------------------------------
plt.figure(figsize=(10, 6))
sns.scatterplot(data=agg, x="avg_speed_mph", y="CalibratedPM25", hue="cluster", palette="tab10")
plt.title("DBSCAN Clusters: Speed vs PM2.5\nClusters help identify patterns in traffic-air quality behavior (e.g., low speed + high PM2.5 = hotspots).")
plt.xlabel("Average Speed (mph)")
plt.ylabel("Calibrated PM2.5")
plt.legend(title="Cluster")
plt.show()

In [ ]:
# CORRELATION MATRIX
# ----------------------------------------
from datascience import Table as DTable
final = DTable.from_df(agg)
final.select("avg_speed_mph", "trip_miles", "trip_seconds", "CalibratedPM25").corr()